In [1]:
%%html
<script>
(function() {
  // Create the toggle button
  const rtlButton = document.createElement("button");
  rtlButton.textContent = "Toggle LTR";
  rtlButton.id = "top-rtl-toggle";
  rtlButton.style.marginLeft = "8px";
  rtlButton.style.padding = "4px 10px";
  rtlButton.style.fontSize = "14px";
  rtlButton.style.cursor = "pointer";

  // State
  var rtlActive = false;

  // Styling function
  var applyStyleToEditor = (editor) => {
    if (!editor) return;
    var direction = getComputedStyle(editor).getPropertyValue('direction')=='rtl' ? 'ltr' : 'rtl';
    var text_align = getComputedStyle(editor).getPropertyValue('text-align')=='right' ? 'left' : 'right';
    editor.style.setProperty('direction', direction, 'important');
    editor.style.setProperty('text-align', text_align, 'important');
  };

  // Toggle logic
  rtlButton.onclick = () => {
    rtlActive = !rtlActive;
    rtlButton.textContent = rtlActive ? "Toggle LTR" : "Toggle RTL";
    document.querySelectorAll('.jp-MarkdownCell .jp-InputArea-editor').forEach(applyStyleToEditor);
    document.querySelectorAll('.jp-RenderedHTMLCommon code, .jp-RenderedHTMLCommon code span').forEach(applyStyleToEditor);
    document.querySelectorAll('jp-RenderedHTMLCommon, .jp-RenderedHTMLCommon *').forEach(applyStyleToEditor);
  };

  // Watch for focus into editing Markdown cells
  // document.addEventListener('focusin', (event) => {
  //   const editor = event.target.closest('.jp-MarkdownCell .jp-InputArea-editor');
  //    if (editor) applyStyleToEditor(editor);
  // });

  // Insert into top toolbar if not already present
  var insertIntoToolbar = () => {
    const toolbar = document.querySelector('.jp-NotebookPanel-toolbar');
    if (toolbar && !document.getElementById("top-rtl-toggle")) {
      toolbar.appendChild(rtlButton);
    } else {
      // Try again in a moment if toolbar isn't ready yet
      setTimeout(insertIntoToolbar, 300);
    }
  };

  insertIntoToolbar();
})();
</script>

In [2]:
%%html
<!-- <style>
  table {display: inline-block}
</style> -->

## קיבוץ וצבירה בנתונים ניסיוניים (Grouping & Aggregation)

בעת ניתוח נתונים פיזיקליים, לעיתים נבצע סדרת מדידות במועדים שונים ובתנאים מגוונים — לדוגמה, מדידת **אנרגיה ממוצעת**, **טמפרטורה**, או **זרם חשמלי** תחת ניסויים שונים.  
באמצעות `groupby` ו־`agg` נוכל לסכם את הנתונים לפי תנאי הניסוי, לחשב ממוצעים, סכומים וסטיות תקן, ואף להוסיף מדדים מחושבים חדשים.

#### קיבוץ בסיסי ו־agg
השיטה `groupby()` מאפשרת לחלק את הנתונים לפי משתנה אחד או יותר (כגון מזהה ניסוי או חודש המדידה)  
ולחשב עבור כל קבוצה מדדים שונים בעזרת `.agg()`:
```python
df.groupby("experiment")["energy_kJ"].agg(["mean", "std", "count"])
```
ניתן גם להגדיר מילון פונקציות:
```python
df.groupby("experiment").agg({"energy_kJ": ["sum", "mean", "count"]})
```
#### שימוש ב־transform

כאשר נרצה לחשב ערך נורמלי לכל שורה — למשל, את אחוז התרומה של מדידה מסוימת לסך האנרגיה של הניסוי —
נשתמש ב־`transform("sum")` כך שהתוצאה תישמר בגודל זהה למספר השורות במקור.

#### ממוצע נע (Rolling / Moving Average)

במדידות רציפות לאורך זמן, יש לעיתים תנודות אקראיות.
ניתן להחליק את הנתונים באמצעות ממוצע נע על פני חלון של מספר ימים או מדידות:
```python
df["energy_smooth"] = df["energy_kJ"].rolling(window=7, min_periods=1).mean()
```
כך מתקבלת מגמה ברורה ונקייה מרעש מדידה.

### תרגילים: קיבוץ, נורמליזציה וחלון נע
ניצור טבלת נתונים שמדמה ניסוי פיזיקלי שבו נמדדה אנרגיה קינטית (בקילו־ג׳ול) לאורך זמן.  
כל מדידה משויכת לניסוי מסוים (A, B או C) ולתאריך ביצוע.  

במהלך התרגילים נבצע שלושה שלבים עיקריים:

1. **קיבוץ נתונים (groupby & agg):**  
   נחלק את המדידות לפי ניסוי וחודש, ונחשב לכל קבוצה את סכום האנרגיה, הממוצע ומספר המדידות.  

2. **נורמליזציה עם transform():**  
   נחשב לכל מדידה את חלקה היחסי בסך האנרגיה של אותו ניסוי — נשתמש ב־`transform("sum")` כדי לשמור את החישוב ברמת השורה.  

3. **חישוב ממוצע נע (rolling mean):**  
   נחשב ממוצע נע של האנרגיה לאורך זמן (7 ימים), כדי לזהות מגמות ולהחליק רעשים במדידות.

תרגילים אלו מדגימים כיצד להשתמש בכלים הסטטיסטיים של Pandas כדי לבצע ניתוח כמותי אמיתי על נתוני ניסוי פיזיקלי.


## קיבוץ וצבירה בנתונים ניסיוניים (Grouping & Aggregation)

בעת ניתוח נתונים פיזיקליים, לעיתים נבצע סדרת מדידות במועדים שונים ובתנאים מגוונים — לדוגמה, מדידת **אנרגיה ממוצעת**, **טמפרטורה**, או **זרם חשמלי** תחת ניסויים שונים.  
באמצעות `groupby` ו־`agg` נוכל לסכם את הנתונים לפי תנאי הניסוי, לחשב ממוצעים, סכומים וסטיות תקן, ואף להוסיף מדדים מחושבים חדשים.

#### קיבוץ בסיסי ו־agg
השיטה `groupby()` מאפשרת לחלק את הנתונים לפי משתנה אחד או יותר (כגון מזהה ניסוי או חודש המדידה)  
ולחשב עבור כל קבוצה מדדים שונים בעזרת `.agg()`:
```python
df.groupby("experiment")["energy_kJ"].agg(["mean", "std", "count"])
```
ניתן גם להגדיר מילון פונקציות:
```python
df.groupby("experiment").agg({"energy_kJ": ["sum", "mean", "count"]})
```
#### שימוש ב־transform

כאשר נרצה לחשב ערך נורמלי לכל שורה — למשל, את אחוז התרומה של מדידה מסוימת לסך האנרגיה של הניסוי —
נשתמש ב־`transform("sum")` כך שהתוצאה תישמר בגודל זהה למספר השורות במקור.

#### ממוצע נע (Rolling / Moving Average)

במדידות רציפות לאורך זמן, יש לעיתים תנודות אקראיות.
ניתן להחליק את הנתונים באמצעות ממוצע נע על פני חלון של מספר ימים או מדידות:
```python
df["energy_smooth"] = df["energy_kJ"].rolling(window=7, min_periods=1).mean()
```
כך מתקבלת מגמה ברורה ונקייה מרעש מדידה.

### תרגילים: קיבוץ, נורמליזציה וחלון נע
ניצור טבלת נתונים שמדמה ניסוי פיזיקלי שבו נמדדה אנרגיה קינטית (בקילו־ג׳ול) לאורך זמן.  
כל מדידה משויכת לניסוי מסוים (A, B או C) ולתאריך ביצוע.  

במהלך התרגילים נבצע שלושה שלבים עיקריים:

1. **קיבוץ נתונים (groupby & agg):**  
   נחלק את המדידות לפי ניסוי וחודש, ונחשב לכל קבוצה את סכום האנרגיה, הממוצע ומספר המדידות.  

2. **נורמליזציה עם transform():**  
   נחשב לכל מדידה את חלקה היחסי בסך האנרגיה של אותו ניסוי — נשתמש ב־`transform("sum")` כדי לשמור את החישוב ברמת השורה.  

3. **חישוב ממוצע נע (rolling mean):**  
   נחשב ממוצע נע של האנרגיה לאורך זמן (7 ימים), כדי לזהות מגמות ולהחליק רעשים במדידות.

תרגילים אלו מדגימים כיצד להשתמש בכלים הסטטיסטיים של Pandas כדי לבצע ניתוח כמותי אמיתי על נתוני ניסוי פיזיקלי.


In [3]:
import pandas as pd
import numpy as np

# --- Setup: simulated physics experiment data ---
np.random.seed(42)

dates = pd.date_range("2025-03-01", periods=60, freq="D")
df = pd.DataFrame({
    "date": np.random.choice(dates, 100),
    "experiment": np.random.choice(["A", "B", "C"], 100, p=[0.4, 0.35, 0.25]),
    "energy_kJ": np.random.uniform(10, 25, 100)
})

df["month"] = df["date"].dt.to_period("M").astype(str)

print("Raw experimental data sample:")
display(df.head())

Raw experimental data sample:


,date,experiment,energy_kJ,month
0,2025-04-08,B,11.212800,2025-04
1,2025-04-21,B,15.544817,2025-04
2,2025-03-29,C,13.632399,2025-03
3,2025-03-15,C,22.047096,2025-03
4,2025-04-12,B,17.054510,2025-04


#### חישוב אנרגיה מצטברת, ממוצע וספירת מדידות לפי ניסוי וחודש

נחשב לכל ניסוי את סכום האנרגיה, הממוצע, ומספר המדידות באותו חודש.


In [4]:
# --- Group by experiment and month ---
energy_summary = (
    df.groupby(["experiment", "month"], as_index=False)
      .agg({"energy_kJ": ["sum", "mean", "count"]})
)
print("Energy summary by experiment and month:")
display(energy_summary)


Energy summary by experiment and month:


experiment    month   energy_kJ                 
                              sum       mean count
0          A  2025-03  362.431879  17.258661    21
1          A  2025-04  392.060613  19.603031    20
2          B  2025-03  386.832572  17.583299    22
3          B  2025-04  371.218332  17.677063    21
4          C  2025-03  197.541725  19.754172    10
5          C  2025-04  121.409078  20.234846     6

#### הוספת אחוז התרומה של כל מדידה לסך האנרגיה בניסוי

נחשב לכל שורה את חלקה היחסי מסך האנרגיה של הניסוי (`energy_kJ / Σexperiment`).  
נשתמש ב־`transform("sum")` כדי לשמור על אורך זהה למקור.


In [5]:
# --- Compute each measurement's share within its experiment ---
df["total_energy_by_exp"] = df.groupby("experiment")["energy_kJ"].transform("sum")
df["share_in_experiment"] = df["energy_kJ"] / df["total_energy_by_exp"]

print("Share of each measurement in its experiment total:")
display(df[["experiment", "energy_kJ", "share_in_experiment"]].head())


Share of each measurement in its experiment total:


,experiment,energy_kJ,share_in_experiment
0,B,11.212800,0.014792
1,B,15.544817,0.020506
2,C,13.632399,0.042741
3,C,22.047096,0.069124
4,B,17.054510,0.022498


#### ממוצע נע של אנרגיה לאורך זמן (7 ימים)

נניח שמדדנו אנרגיה יומית לאורך חודשיים ורוצים להחליק תנודות יומיות.  
נחשב ממוצע נע ל־7 ימים עבור כל ניסוי בנפרד.


In [6]:
# --- Compute 7-day rolling mean of energy for each experiment ---
# Sort for proper rolling within each experiment
df_sorted = df.sort_values(["experiment", "date"]).set_index("date")

# Apply rolling mean per experiment
df_sorted["energy_rolling_7d"] = (
    df_sorted.groupby("experiment")["energy_kJ"]
    .transform(lambda x: x.rolling(window=7, min_periods=1).mean())
)

print("7-day rolling mean per experiment:")
display(df_sorted[["experiment", "energy_kJ", "energy_rolling_7d"]].head(15))


7-day rolling mean per experiment:


,experiment,energy_kJ,energy_rolling_7d
date,,,
2025-03-02,A,18.121720,18.121720
2025-03-02,A,10.654057,14.387888
2025-03-04,A,12.638879,13.804885
2025-03-06,A,24.918258,16.583228
2025-03-07,A,17.301132,16.726809
2025-03-09,A,20.814099,17.408024
2025-03-10,A,11.130194,16.511191
2025-03-11,A,20.437192,16.841973
2025-03-11,A,22.875382,18.587877


`````{admonition} סיכום
:class: tip
- השתמשנו ב־`groupby()` ו־`agg()` כדי לחשב מדדים מרכזיים (סכום, ממוצע, ספירה) לכל ניסוי ולכל חודש.  
- בעזרת `transform("sum")` חישבנו לכל שורה את אחוז התרומה שלה לסך המדידות של אותו ניסוי.  
- הדגמנו שימוש ב־`rolling(window=7)` כדי לחשב ממוצע נע על פני שבוע במדידות יומיות, ולזהות מגמות לאורך זמן.  
- כל הדוגמאות מתבססות על הקשרים פיזיקליים — ניתוח אנרגיה, ניסויים חוזרים, ומגמות בזמן.
`````

In [7]:
import json
from jupyterquiz import display_quiz

quiz_json = '''
[
  {
    "question": "במסגרת ניתוח ניסוי פיזיקלי נמדדה אנרגיה יומית (kJ) עבור מספר ניסויים שונים. אנו רוצים לחשב:<br><br>1️⃣ את סכום וממוצע האנרגיה לכל ניסוי,<br>2️⃣ את אחוז התרומה של כל מדידה לסך הניסוי,<br>3️⃣ ואת הממוצע הנע ל־7 ימים כדי להחליק תנודות.<br><br>אילו פקודות של Pandas מתאימות לכל אחת מהמשימות?",
    "type": "many_choice",
    "answers": [
      {
        "answer": "groupby() + agg(), transform('sum'), rolling(window=7)",
        "correct": true,
        "feedback": "נכון! groupby/agg מבצעים צבירה לפי ניסוי, transform משמש לנורמליזציה ברמת השורה, ו־rolling(window=7) מחשב ממוצע נע."
      },
      {
        "answer": "pivot_table(), merge(), melt()",
        "correct": false,
        "feedback": "לא מדויק — אלו פונקציות לעיצוב ושינוי מבנה הנתונים, לא לחישוב צבירות או ממוצעים נעים."
      },
      {
        "answer": "apply(lambda x: ...), concat(), join()",
        "correct": false,
        "feedback": "לא — apply יעבוד אך פחות יעיל, ו־concat/join אינם קשורים לחישובים קבוצתיים או ממוצעים נעים."
      },
      {
        "answer": "unique(), drop_duplicates(), isin()",
        "correct": false,
        "feedback": "לא — אלו פונקציות לסינון וזיהוי רשומות ייחודיות, לא לניתוחים כמותיים או חישובים סטטיסטיים."
      }
    ]
  }
]
'''

myquiz = json.loads(quiz_json)
display_quiz(myquiz)


<IPython.core.display.Javascript object>